# week 1 homework

In [32]:
# imports
!pip install statsmodels
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm
from sklearn.neighbors import NearestNeighbors


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
hw1_df = pd.read_csv('homework_1.1.csv')
hw1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1000 non-null   int64  
 1   X1          1000 non-null   float64
 2   X2          1000 non-null   float64
 3   X3          1000 non-null   float64
 4   Y           1000 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 39.2 KB


In [5]:
hw1_df.describe()

,Unnamed: 0,X1,X2,X3,Y
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,499.500000,0.045555,0.030001,0.057442,0.278379
std,288.819436,2.191768,0.981732,1.050014,5.201346
min,0.000000,-6.163467,-2.703008,-3.348274,-13.862793
25%,249.750000,-1.465901,-0.662930,-0.677971,-3.396553
50%,499.500000,0.039807,0.001803,0.029222,0.044328
75%,749.250000,1.502296,0.688336,0.768379,3.694404
max,999.000000,8.464936,3.153685,3.871509,18.146133


In [6]:
hw1_df.head()

,Unnamed: 0,X1,X2,X3,Y
0,0,-0.440646,-0.390227,0.156718,-0.877671
1,1,-3.810099,-1.304665,-1.105117,-10.130388
2,2,-1.425451,-0.340049,1.115908,0.284068
3,3,-1.325750,0.161906,-0.254670,-1.994344
4,4,3.120263,1.487343,-1.164839,2.030030


### linear regression questions

In [34]:
# features
X = hw1_df[['X1', 'X2', 'X3']]

# target
y = hw1_df['Y']

# initialize model
model = LinearRegression()

# fit model
model.fit(X, y)

# coefficients and intercept
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)

# prediction
y_pred = model.predict(X)

# evaluate performance
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)


Intercept: 0.0026430033841688405
Coefficients: [1.00713766 1.96456859 2.97548854]
Mean Squared Error: 0.2509162964680301
R^2 Score: 0.9907160704940021


In [37]:
# df to clearly see coefficient for each feature
coeff_df = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print(coeff_df)

    Coefficient
X1     1.007138
X2     1.964569
X3     2.975489


In [38]:
# answer question 2...
# regress Y on each Xi alone (univariate)
# and compare those coefficients to the ones from the multiple regression above

# --- Multivariate Regression (Y on X1, X2, X3) ---
features = ['X1', 'X2', 'X3']
X = hw1_df[features]
y = hw1_df['Y']

multi_model = LinearRegression()
multi_model.fit(X, y)

print("--- Model Coefficients (from Y ~ X1 + X2 + X3) ---")
print(f'Intercept: {multi_model.intercept_:.4f}')

# initial df
coeff_df = pd.DataFrame(multi_model.coef_, features, columns=['Coefficient_Multivariate'])
print(coeff_df)

# --- Univariate Regressions (Y on each Xi) ---
print("--- Univariate vs. Multivariate Coefficient Analysis ---")

# empty list to store the regression coefficients
univariate_coeffs = []
for feature in features:
    # select a single feature
    X_single = hw1_df[[feature]]

    # train linear regression model
    single_model = LinearRegression()
    single_model.fit(X_single, y)

    # append the coefficient to the list
    univariate_coeffs.append(single_model.coef_[0])

# add the new coefficients as a new column in df
coeff_df['Coefficient_Univariate'] = univariate_coeffs

# --- Comparison and Conclusion ---
coeff_df['Difference'] = abs(coeff_df['Coefficient_Multivariate'] - coeff_df['Coefficient_Univariate'])

print("\n--- Comparison of Coefficients ---")
print(coeff_df)

# find and print the feature with the greatest difference
greatest_diff_feature = coeff_df['Difference'].idxmax()
greatest_diff_value = coeff_df['Difference'].max()

print(f"\nThe feature with the greatest difference is '{greatest_diff_feature}' with a difference of {greatest_diff_value:.4f}.")



--- Model Coefficients (from Y ~ X1 + X2 + X3) ---
Intercept: 0.0026
    Coefficient_Multivariate
X1                  1.007138
X2                  1.964569
X3                  2.975489
--- Univariate vs. Multivariate Coefficient Analysis ---

--- Comparison of Coefficients ---
    Coefficient_Multivariate  Coefficient_Univariate  Difference
X1                  1.007138                1.841761    0.834623
X2                  1.964569                4.083613    2.119044
X3                  2.975489                3.097041    0.121553

The feature with the greatest difference is 'X2' with a difference of 2.1190.


In [39]:
# t stat

# Features and target
X = hw1_df[['X1', 'X2', 'X3']]
y = hw1_df['Y']

# Add constant (intercept)
X = sm.add_constant(X)

# Fit OLS regression
model = sm.OLS(y, X).fit()

# Print summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                 3.543e+04
Date:                Mon, 01 Sep 2025   Prob (F-statistic):               0.00
Time:                        19:01:35   Log-Likelihood:                -727.62
No. Observations:                1000   AIC:                             1463.
Df Residuals:                     996   BIC:                             1483.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0026      0.016      0.166      0.8

In [61]:
# KNN
# questions 4-5

df = pd.read_csv("homework_1.2.csv")

# Split
X0 = df[df["X"]==0].reset_index(drop=True)
X1 = df[df["X"]==1].reset_index(drop=True)

# Fit NN on X0, query with X1
nn = NearestNeighbors(n_neighbors=1)
nn.fit(X0[["Z"]])
distances, indices = nn.kneighbors(group1[["Z"]])

# Build match DataFrame
matches = pd.DataFrame({
    "X1_index": group1.index,  # index of X=1 row
    "X1_Z": group1["Z"].values,
    "BestMatch_X0_index": group0.index[indices.flatten()],  # matched X=0 index
    "BestMatch_X0_Z": group0.iloc[indices.flatten()]["Z"].values,
    "Distance": distances.flatten()
})

# Show top matches
print(matches.head())


   X1_index      X1_Z  BestMatch_X0_index  BestMatch_X0_Z  Distance
0         0  0.715189                  48        0.716327  0.001138
1         1  0.645894                  25        0.653108  0.007214
2         2  0.437587                  18        0.437032  0.000555
3         3  0.891773                   9        0.778157  0.113616
4         4  0.963663                   9        0.778157  0.185506


In [70]:
# KNN
# question 5

effect = X1["Y"].mean() - matched_X0["Y"].mean()
print(effect)

0.5433600651913855


In [67]:
# questions 6-7
# Load dataset
df = pd.read_csv("homework_1.2.csv")

# Split into groups
X0 = df[df["X"] == 0].reset_index(drop=True)
X1 = df[df["X"] == 1].reset_index(drop=True)

# KNN

Z0 = X0[["Z"]].values
Z1 = X1[["Z"]].values

nn = NearestNeighbors(radius=0.2)
nn.fit(Z0)

# find all neighbors of X=1 within distance 0.2
distances, indices = nn.radius_neighbors(Z1)


In [68]:
# flatten all matched X0 indices
all_matches = [j for idxs in indices for j in idxs]

# count duplicates: total matches minus unique matches
num_duplicates = len(all_matches) - len(set(all_matches))
print("Number of duplicates:", num_duplicates)


Number of duplicates: 685


In [69]:
# Q7 
# compute mean Y for each neighbor group
matched_means = []
for idxs in indices:
    if len(idxs) > 0:
        matched_means.append(X0.iloc[idxs]["Y"].mean())
    else:
        matched_means.append(np.nan)  # if no neighbor, optional: skip or NaN

# average across all X=1 (ignore NaNs)
avg_matched_y = np.nanmean(matched_means)

# mean Y for X=1
avg_y_x1 = X1["Y"].mean()

# effect
effect = avg_y_x1 - avg_matched_y
print("Effect:", effect)


Effect: 0.5844124774008066
